In [2]:
// Nuget Packages
#r "nuget: MelonChart.NET, 2.*"

#r "nuget: Microsoft.SemanticKernel, 1.*"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.*"
#r "nuget: Microsoft.SemanticKernel.Core, 1.*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.*-*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.*-*"

#r "nuget: System.Linq.Async, 6.*"

Installed Packages MelonChart.NET, 2.0.0 Microsoft.SemanticKernel, 1.16.1 Microsoft.SemanticKernel.Connectors.OpenAI, 1.16.1 Microsoft.SemanticKernel.Core, 1.16.1 Microsoft.SemanticKernel.Plugins.Core, 1.16.1-alpha Microsoft.SemanticKernel.Plugins.Memory, 1.16.1-alpha System.Linq.Async, 6.0.1

In [3]:
// Add using statements
using System.ComponentModel;
using System.Net.Http;
using System.Text.Encodings.Web;
using System.Text.Json;
using System.Text.Json.Serialization;

using MelonChart.Models;

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;

using Kernel = Microsoft.SemanticKernel.Kernel;

In [4]:
// Azure OpenAI configurations
var endpoint = "<endpoint-placeholder>";
var apiKey = "<apikey-placeholder>";
var deploymentName = "<deploymentname-placeholder>";

In [5]:
// Build Semantic Kernel
var kernel = Kernel.CreateBuilder()
                   .AddAzureOpenAIChatCompletion(
                       deploymentName: deploymentName,
                       endpoint: endpoint,
                       apiKey: apiKey)
                   .Build();

var prompts = kernel.ImportPluginFromPromptDirectory("Prompts");

// Import codes
#!import Plugins/AddMemory/AddMelonChartPlugin.cs

// Import plugins
kernel.ImportPluginFromType<AddMelonChartPlugin>();

In [6]:
// Build Semantic Memory
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0010
#pragma warning disable SKEXP0050

var memory = new MemoryBuilder()
                 .WithAzureOpenAITextEmbeddingGeneration(
                     deploymentName: "model-textembeddingada002-2",
                     endpoint: endpoint,
                     apiKey: apiKey)
                 .WithMemoryStore(new VolatileMemoryStore())
                 .Build();

In [7]:
// Add HttpClient instance.
var http = new HttpClient();

In [8]:
// Add JsonSerializerOptions instance.
var jso = new JsonSerializerOptions()
{
    WriteIndented = false,
    PropertyNamingPolicy = JsonNamingPolicy.CamelCase,
    Encoder = JavaScriptEncoder.UnsafeRelaxedJsonEscaping,
    Converters = { new JsonStringEnumConverter(JsonNamingPolicy.CamelCase) },
};

In [9]:
var question = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("무엇이 궁금한가요?");

Console.WriteLine($"User: {question}");

User: 에스파 현재 랭킹이 몇위야?


In [10]:
var intent = await kernel.InvokeAsync<string>(
                        function: prompts["GetIntent"],
                        arguments: new KernelArguments()
                        {
                            { "input", question }
                        });

Console.WriteLine(intent);

CurrentRank


In [11]:
// Invoke the prompt - RefineQuestion
var refined = await kernel.InvokeAsync<string>(
                        function: prompts["RefineQuestion"],
                        arguments: new KernelArguments()
                        {
                            { "input", question },
                            { "intent", intent }
                        });

Console.WriteLine(refined);

What is the rank of the song, 에스파, in the chart?


In [12]:
// Invoke the plugin - Add Melon Chart data
await kernel.InvokeAsync(
    pluginName: nameof(AddMelonChartPlugin),
    functionName: nameof(AddMelonChartPlugin.AddChart), 
    arguments: new KernelArguments()
    {
        { "memory", memory }, 
        { "http", http },
        { "jso", jso }, 
    }
);

- Stored: aespa - Supernova
- Stored: NewJeans - How Sweet
- Stored: 이영지 - Small girl (feat. 도경수(D.O.))
- Stored: (여자)아이들 - 클락션 (Klaxon)
- Stored: NewJeans - Supernatural
- Stored: KISS OF LIFE - Sticky
- Stored: 이클립스 (ECLIPSE) - 소나기
- Stored: QWER - 고민중독
- Stored: NewJeans - Bubble Gum
- Stored: DAY6 (데이식스) - 한 페이지가 될 수 있게
- Stored: (여자)아이들 - 나는 아픈 건 딱 질색이니까
- Stored: aespa - Armageddon
- Stored: 아일릿(ILLIT) - Magnetic
- Stored: IVE (아이브) - 해야 (HEYA)
- Stored: 지코 (ZICO) - SPOT! (feat. JENNIE)
- Stored: TWS (투어스) - 첫 만남은 계획대로 되지 않아
- Stored: DAY6 (데이식스) - Welcome to the Show
- Stored: DAY6 (데이식스) - 예뻤어
- Stored: 이창섭 - 천상연
- Stored: 이무진 - 에피소드
- Stored: Crush - 미안해 미워해 사랑해
- Stored: RIIZE - Boom Boom Bass
- Stored: NewJeans - Hype Boy
- Stored: 아이유 - Love wins all
- Stored: 임재현 - 비의 랩소디
- Stored: 순순희 (지환) - 슬픈 초대장
- Stored: IVE (아이브) - I AM
- Stored: 너드커넥션 (Nerd Connection) - 그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection))
- Stored: TWS (투어스) - 내가 S면 넌 나의 N이 되어줘
- Stored: 비비 (BIBI) - 밤양갱
- St

In [13]:
// Invoke the plugin - Find songs
var results = await kernel.InvokeAsync(
    pluginName: nameof(AddMelonChartPlugin),
    functionName: nameof(AddMelonChartPlugin.FindSongs), 
    arguments: new KernelArguments()
    {
        { "memory", memory }, 
        { "question", refined },
        { "jso", jso }, 
    }
);

var data = results.GetValue<List<ChartItem>>().Select(p => JsonSerializer.Serialize(p, jso)).Aggregate((x, y) => $"{x}\n{y}");

Console.WriteLine(data);

{"songId":"37524037","rank":1,"rankStatus":"none","rankStatusValue":0,"title":"Supernova","artist":"aespa","album":"Armageddon - The 1st Album","image":"https://cdnimg.melon.co.kr/cm2/album/images/114/87/023/11487023_20240527154018_500.jpg/melon/resize/120/quality/80/optimize","trackId":null,"trackUri":null,"danceability":null,"valence":null}
{"songId":"37524038","rank":12,"rankStatus":"none","rankStatusValue":0,"title":"Armageddon","artist":"aespa","album":"Armageddon - The 1st Album","image":"https://cdnimg.melon.co.kr/cm2/album/images/114/87/023/11487023_20240527154018_500.jpg/melon/resize/120/quality/80/optimize","trackId":null,"trackUri":null,"danceability":null,"valence":null}
{"songId":"36956630","rank":41,"rankStatus":"none","rankStatusValue":0,"title":"Drama","artist":"aespa","album":"Drama - The 4th Mini Album","image":"https://cdnimg.melon.co.kr/cm2/album/images/113/62/544/11362544_20231110142622_500.jpg/melon/resize/120/quality/80/optimize","trackId":null,"trackUri":null,"d

In [14]:
// Invoke the prompt - RefineResult
var refined = await kernel.InvokeAsync<string>(
                        function: prompts["RefineResult"],
                        arguments: new KernelArguments()
                        {
                            { "input", data },
                            { "intent", intent }
                        });

Console.WriteLine(refined);

aespa|1|Supernova|Armageddon - The 1st Album
aespa|12|Armageddon|Armageddon - The 1st Album
aespa|41|Drama|Drama - The 4th Mini Album
aespa|44|Spicy|MY WORLD - The 3rd Mini Album
이클립스 (ECLIPSE)|7|소나기|선재 업고 튀어 OST Part 1
순순희(기태)|98|그대가 내 안에 박혔다(그내박)|그대가 내 안에 박혔다(그내박)
AKMU (악뮤)|64|어떻게 이별까지 사랑하겠어, 널 사랑하는 거지|항해
멜로망스|43|사랑인가 봐|사랑인가 봐 (사내맞선 OST 스페셜 트랙)
이무진|20|에피소드|에피소드
부석순 (SEVENTEEN)|51|파이팅 해야지 (Feat. 이영지)|부석순 1st Single Album 'SECOND WIND'
경서예지|66|다정히 내 이름을 부르면|다정히 내 이름을 부르면 (경서예지 x 전건호)
박재정|35|헤어지자 말해요|1집 Alone
엔플라잉 (N.Flying)|62|네가 내 마음에 자리 잡았다 (Into You)|네가 내 마음에 자리 잡았다
LE SSERAFIM (르세라핌)|78|EASY|EASY
폴킴|59|모든 날, 모든 순간 (Every day, Every Moment)|'키스 먼저 할까요?' OST Part.3
유회승 (엔플라잉)|54|그랬나봐|선재 업고 튀어 OST Part 6
PLAVE|97|우리 영화|ASTERUM : 134-1
DAY6 (데이식스)|18|예뻤어|Every DAY6 February
TWS (투어스)|29|내가 S면 넌 나의 N이 되어줘|TWS 2nd Mini Album ‘SUMMER BEAT!’
임영웅|81|모래 알갱이|모래 알갱이
지코 (ZICO)|15|SPOT! (feat. JENNIE)|SPOT!
DK(디셈버)|94|심(心)|심(心)
TWS (투어스)|16|첫 만남은 계획대로 되지 않아|TWS 1st Mini Album ‘Sparkling Blue’
tr